<a href="https://colab.research.google.com/github/george-zakharov/ml_notebooks/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 9



#### Light: 

Выполнить задачу text summarization для своего набора статей на английском языке.


#### Pro: 

Выполнить задачу text summarization для своего набора статей на русском языке. 
В качестве векторных представлений предложений использовать либо предобученные 
модели https://rusvectores.org/ru/models/ или собственный embending, 
полученный из SVD разложения.

# Load libs

In [38]:
from google.colab import files
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
import networkx as nx
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from numba import jit

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Light (News dataset from Kaggle)

## Load dataset

In [2]:
!pip install kaggle

In [3]:
file = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle

In [5]:
!kaggle datasets download -d asad1m9a9h6mood/news-articles # https://www.kaggle.com/asad1m9a9h6mood/news-articles

  0% 0.00/1.73M [00:00<?, ?B/s]
100% 1.73M/1.73M [00:00<00:00, 58.2MB/s]


In [6]:
!unzip news-articles.zip

Archive:  news-articles.zip
  inflating: Articles.csv            


In [0]:
df = pd.read_csv('Articles.csv', engine = 'python')

## User defined functions

In [0]:
# function to remove stopwords
@jit
def remove_stopwords(sentence):
    sen_new = " ".join([i for i in sentence if i not in stop_words])
    return sen_new

## Dataset exploration

In [9]:
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [10]:
df.Article[0]

'KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG). Bukhari said Karachi transporters will cut fares when decrease in CNG prices will be made.                        \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n'

## Data transformation

In [0]:
# split the the text in the articles into sentences
sentences = []
for article in df.Article:
  sentences.append(sent_tokenize(article))  

In [70]:
sentences[0]

['KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG).',
 'Bukhari said Karachi transporters will cut fares when decrease in CNG prices will be made.']

In [0]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [72]:
sentences[0]

'KARACHI: The Sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, Geo News reported.Sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.Meanwhile, Karachi Transport Ittehad (KTI) has refused to abide by the government decision.KTI President Irshad Bukhari said the commuters are charged the lowest fares in Karachi as compare to other parts of the country, adding that 80pc vehicles run on Compressed Natural Gas (CNG).'

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [sentence.lower() for sentence in clean_sentences]

In [74]:
clean_sentences[0]

'karachi  the sindh government has decided to bring down public transport fares by   per cent due to massive reduction in petroleum product prices by the federal government  geo news reported sources said reduction in fares will be applicable on public transport  rickshaw  taxi and other means of traveling meanwhile  karachi transport ittehad  kti  has refused to abide by the government decision kti president irshad bukhari said the commuters are charged the lowest fares in karachi as compare to other parts of the country  adding that   pc vehicles run on compressed natural gas  cng  '

In [0]:
stop_words = stopwords.words('english')

In [0]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [19]:
clean_sentences[0]

'karachi sindh government decided bring public transport fares per cent due massive reduction petroleum product prices federal government geo news reported sources said reduction fares applicable public transport rickshaw taxi means traveling meanwhile karachi transport ittehad kti refused abide government decision kti president irshad bukhari said commuters charged lowest fares karachi compare parts country adding pc vehicles run compressed natural gas cng'

In [20]:
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Extract word vectors
word_embeddings = {}

f = open('/content/drive/My Drive/Colab Notebooks/ML/9/glove.6B.100d.txt', encoding = 'utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    word_embeddings[word] = coefs
f.close()

In [22]:
word_embeddings['sport']

array([ 0.11135  ,  0.41816  ,  0.21722  , -0.042453 , -0.5187   ,
        0.12853  ,  0.44858  ,  0.75437  , -0.041852 ,  0.079843 ,
       -0.023021 , -0.2399   ,  0.1317   , -0.39341  , -0.96106  ,
       -0.17475  ,  0.74805  ,  0.43138  , -0.18697  ,  0.75033  ,
        0.070473 ,  0.41777  , -0.049083 , -0.064871 ,  0.543    ,
        0.062025 , -0.45834  , -0.58207  , -0.053786 ,  0.4619   ,
       -0.36932  ,  0.51805  , -0.99383  , -0.54979  ,  0.7254   ,
        0.57203  , -0.43165  ,  0.90363  , -1.0984   , -0.14603  ,
       -0.20707  , -0.14062  ,  0.052919 , -0.82595  , -0.6578   ,
        0.40092  ,  0.90385  , -0.0052617, -0.0045023, -0.87071  ,
       -0.24242  ,  0.28286  , -0.91073  ,  0.85632  ,  0.20688  ,
       -1.5929   , -0.37628  ,  1.1513   ,  1.8729   ,  0.18253  ,
       -0.063582 ,  0.30018  , -0.39956  ,  0.34561  ,  0.63868  ,
        0.35972  , -0.017979 , -0.52348  ,  0.56879  ,  0.39967  ,
        0.89808  , -0.69885  ,  0.23404  ,  0.28625  ,  0.0527

In [0]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()]) / (len(i.split()) + 0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [81]:
len(sentence_vectors)

13587

In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences[:2000]), len(sentences[:2000])])

In [0]:
for i in range(len(sentences[:2000])):
    for j in range(len(sentences[:2000])):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1, 100), sentence_vectors[j].reshape(1, 100))[0, 0]

In [0]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [0]:
ranked_sentences = sorted(((scores[i], s) for i,s in enumerate(sentences[:2000])), reverse = True)

In [120]:
len(ranked_sentences)

2000

In [121]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])
    print('')

Inflation in South Asia s second largest economyslowed each month this year through April as transport and foodprices fell prompting the central bank to cut the benchmarkinterest rate in May to the lowest level in 42 years.It said Moody�s Investors Service upgraded Pakistan�ssovereign credit ratings for the first time since 2008 in June butsaid stalling the ongoing IMF program or an unstable politicalenvironment would be credit negative.The report noted that China was standing by Pakistan as thenation�s long time strategic ally.In April, Asia s largest economy, China signed deals for $28billion of investments in Pakistan as part of a planned $45 billioneconomic corridor that includes power plants and dams.The development in cities and smaller towns is trickling downand is good news for smaller contractors as well the report said.�Business has been very good and there s no doubt my work hastripled in five years,� the Bloomberg report quoted Mohammed HassanBakshi 43 a builder in Karachi.

In [122]:
text = 'US data has not been exceptionally strong, and thus, could mean that oil prices could be facing some headwinds towards the end of the week," Ang said.In Asian trade, US benchmark West Texas Intermediate for November delivery was down 10 cents to $47.16 and Brent crude for December fell 12 cents to $50.34 a barrel at around 0300 GMT.Both contracts fell for four straight trading days last week before advancing Friday on the back of data showing a fresh decline in US oil exploration, which suggested lower output that could help ease the global oversupply.Sanjeev Gupta, head of the Asia Pacific oil and gas practice at professional services firm EY, said the market will also be seeking clues from a meeting of OPEC and non-OPEC technical experts in Vienna on Wednesday.Oil-producer Russia said last week it was prepared to discuss output cuts during the meeting.'
kw = keywords(text)
print(kw)

oil
fell
trade
trading
output
services
technical
opec
texas
west
oversupply
gupta


# Useful links

* https://www.kaggle.com/asad1m9a9h6mood/news-articles